# Libraries

In [1]:
# %pip install -U transformers
# %pip install -U datasets
# %pip install -U accelerate
# %pip install -U peft
# %pip install -U trl
# %pip install -U bitsandbytes

In [1]:
import os, torch, wandb

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)

from datasets import load_dataset, concatenate_datasets
from trl import SFTTrainer, setup_chat_format
from dataclasses import dataclass

C:\Users\USER_ELISEY\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
file_with_tokens = "tokens.txt"
model_casual_name = "Qwen 1.5B"
path_to_source_model = "C:\\Users\\USER_ELISEY\\qwen2.5_1.5"
path_to_save_finetuned_model = "russia_chad_1.5"

path_to_dataset = "miracl/miracl"
dataset_casual_name = "MiraCl Russian dataset"

'''
If paths don't work, please try local except of global path(or reversed)
'''

"\nIf paths don't work, please try local except of global path(or reversed)\n"

## Setup Huggingface 🤗 & Wandb

In [4]:
from huggingface_hub import login

f = open(file_with_tokens, "r")
hf_t = f.readline()[:-1]
wndb_t = f.readline()[:-1]

login(token = hf_t)

wandb.login(key = wndb_t)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\USER_ELISEY\.cache\huggingface\token
Login successful


wandb: Currently logged in as: ez1071 (ez1071-mipt). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\USER_ELISEY\_netrc


True

In [5]:
run = wandb.init(
    project=f'Fine-tune {model_casual_name} on Russian Dataset', 
    job_type="training"
)

In [6]:
@dataclass
class Config:
    model_name = model_casual_name
    dataset_name = dataset_casual_name
    new_model = path_to_save_finetuned_model
    torch_dtype = torch.float16
    attn_implementation = "eager"
cfg = Config()

# Loading model and tokenizer

In [7]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=cfg.torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    path_to_source_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=cfg.attn_implementation
)
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear4bit(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear4bit(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
    

In [8]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(path_to_source_model)
tokenizer.padding_side = 'right'
tokenizer.padding_token = '<|pad|>'
print(len(tokenizer))

151665


## LoRA adapter

In [9]:
# LoRA config
peft_config = LoraConfig(
    r=32
    ,#16
    lora_alpha=64,#64
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

# Data

## Load

In [10]:
dataset = load_dataset(path_to_dataset, 'ru', trust_remote_code=True)

In [11]:
# dataset["train"]["positive_passages"]

## Format to chat 

In [12]:
def format_chat_template(row):
    content = row["positive_passages"][0]["text"] + ". "
    for i in range(1, len(row["positive_passages"])):
        content += row["positive_passages"][i]["text"]
        content += ". "
    row_json = [{"role": "user", "content": row["query"]},
               {"role": "assistant", "content": content}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

In [13]:
dataset = concatenate_datasets([
    dataset['dev'],
    dataset['train']
])

dataset = dataset.remove_columns('negative_passages')

In [14]:
dataset['positive_passages'][0][0]["text"]

'Кари́бский кризис\xa0— исторический термин, определяющий чрезвычайно напряжённое политическое, дипломатическое и военное противостояние между Советским Союзом и Соединёнными Штатами в октябре 1962 года, которое было вызвано размещением США ядерного оружия в Турции в 1961 году и впоследствии тайной переброской и размещением на Кубе военных частей и подразделений Вооружённых Сил СССР, техники и вооружения, включая ядерное оружие. Кризис мог привести к глобальной ядерной войне. Кубинцы называют его «Октябрьским кризисом» (), в США распространено название «Кубинский ракетный кризис» ().'

In [15]:
dataset = dataset.map(
    format_chat_template,
    num_proc=1,
)

Map: 100%|██████████| 5935/5935 [00:00<00:00, 6299.91 examples/s]


## Select only part

In [16]:
dataset_sh = dataset.shuffle(seed=2024)#.select(range(10_000))
dataset_sh

Dataset({
    features: ['query_id', 'query', 'positive_passages', 'text'],
    num_rows: 5935
})

In [17]:
dataset_sh = dataset_sh.train_test_split(0.1)

In [18]:
dataset_sh = dataset_sh.remove_columns('positive_passages')
dataset_sh = dataset_sh.remove_columns('query')
dataset_sh = dataset_sh.remove_columns('query_id')

In [19]:
dataset_sh

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 5341
    })
    test: Dataset({
        features: ['text'],
        num_rows: 594
    })
})

# Train model

## Training arguments

In [20]:
training_arguments = TrainingArguments(
    output_dir=cfg.new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
#     num_train_epochs=1,
    max_steps=5000,
    eval_strategy="steps",
    eval_steps=200,
    logging_steps=20,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    group_by_length=True,
    report_to="wandb",
    run_name="Llama-3.1-medicine",
)

In [21]:
print(len(tokenizer))

151665


## Train model

In [22]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_sh["train"],
    eval_dataset=dataset_sh["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

C:\Users\USER_ELISEY\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
C:\Users\USER_ELISEY\AppData\Roaming\Python\Python311\site-packages\trl\trainer\sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
C:\Users\USER_ELISEY\AppData\Roaming\Python\Python311\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Map: 100%|██████████| 594/594 [00:00<00:00, 2795.89 examples/s]
max_steps is given, it will o

In [23]:
trainer.train()

  0%|          | 20/5000 [00:14<57:12,  1.45it/s] 

{'loss': 1.9043, 'grad_norm': 1.0541912317276, 'learning_rate': 0.0001995991983967936, 'epoch': 0.01}


  1%|          | 40/5000 [00:28<56:16,  1.47it/s]

{'loss': 1.722, 'grad_norm': 2.072873115539551, 'learning_rate': 0.00019879759519038077, 'epoch': 0.01}


  1%|          | 60/5000 [00:42<56:43,  1.45it/s]

...

                                                     
100%|██████████| 5000/5000 [1:33:04<00:00,  1.46it/s]

{'eval_loss': 1.4753975868225098, 'eval_runtime': 85.2535, 'eval_samples_per_second': 6.967, 'eval_steps_per_second': 6.967, 'epoch': 1.87}


100%|██████████| 5000/5000 [1:33:05<00:00,  1.12s/it]

{'train_runtime': 5585.0477, 'train_samples_per_second': 1.79, 'train_steps_per_second': 0.895, 'train_loss': 1.3976033634185792, 'epoch': 1.87}


TrainOutput(global_step=5000, training_loss=1.3976033634185792, metrics={'train_runtime': 5585.0477, 'train_samples_per_second': 1.79, 'train_steps_per_second': 0.895, 'total_flos': 2.956069404358963e+16, 'train_loss': 1.3976033634185792, 'epoch': 1.872308556450103})

In [24]:
path_to_save = path_to_save_finetuned_model + "\\final"
trainer.save_model(path_to_save)
model.save_pretrained(path_to_save)
tokenizer.save_pretrained(path_to_save)

('russia_chad_1.5\\final\\tokenizer_config.json',
 'russia_chad_1.5\\final\\special_tokens_map.json',
 'russia_chad_1.5\\final\\vocab.json',
 'russia_chad_1.5\\final\\merges.txt',
 'russia_chad_1.5\\final\\added_tokens.json',
 'russia_chad_1.5\\final\\tokenizer.json')